In [58]:
import numpy as np
import math
from collections import defaultdict

In [59]:
def Exponential_distribution(lambdaValue):
    j1 = np.random.uniform(0, 1)
    if (j1 == 0): j1 += 0.0001
    j2 = -math.log(j1)/lambdaValue
    return j2

In [60]:
def Normal_distribution(mean,stdev):   
    #TO MODEL BASED ON CUMULATIVE DENSITY FUNCTION OF NORMAL DISTRIBUTION BASED ON BOOK OF SHELDON ROSS, Simulation, The polar method, p80.
    while(t>=1 or t==0):
        r1 = np.random.uniform(0, 1)*2 - 1 #randomNumber 1
        r2 = np.random.uniform(0, 1)*2 - 1
        t=r1*r1+r2*r2
    
    multiplier = math.sqrt(-2*math.log(t)/t)
    x = r1 * multiplier * stdev + mean
    return x   

In [61]:
def initialize_functions(): #Put all variables to zero
    ### INPUT DATA RELATED TO SYSTEM JOBS ###
    max_C= 20000
    for i1 in range(0,max_C):
        current_station[i1] = 0
    
    ### VARIABLES RELATED TO system SCANS ###
    for i1 in range(0,nr_stations):
        n_ws[i1] = 0
    
    for i2 in range(0,K):
        mean_customers_system[i2] = 0
        tot_n[i2] = 0
        for i1 in range(0,nr_stations):
            tot_n_ws[i2][i1] = 0
    
    ### PARAMETERS RELATED TO ARRIVAL OF SCANS ###
    max_AS= 5
    for i1 in range(0,nr_stations):
        n_a_ws[i1] = 0

    for i2 in range(0,K):
        mean_interarrival_time[i2] = 0
        for i6 in range(0,max_AS):
            tot_lambda[i2][i6] = 0

        
        for i3 in range(0, max_C):
            time_arrival[i2][i3] = 0
            for i1 in range(0, nr_stations):
                time_arrival_ws[i2][i1][i3] = 0
            scan_type[i3] = 0
            
    for i6 in range(0,max_AS):
            t_a[i6] = 0
    
    ### PARAMETERS RELATED TO Processing OF SCANS ###
    for i1 in range(0,nr_stations):
        n_d_ws[i1] = 0
        for i6 in range(0,nr_servers[i1]):
            t_d[i1][i6] = 0
            current_cust[i1][i6] = 0
        for i6 in range(0, max_C):
            list_scan[i1][i6] = -1

    for i2 in range(0,K):
        mean_service_time[i2] = 0
        tot_mu[i2] = 0;
        for i1 in range(0,nr_stations):
            for i3 in range(0,max_C):
                time_service[i2][i1][i3] = 0
        
    ### PARAMETERS RELATED TO waiting OF SCANS ###
    for i2 in range(0,K):
        mean_waiting_time[i2] = 0
        waiting_time[i2] = 0
        mean_customers_queue[i2] = 0
        tot_n_queue[i2] = 0
        for i1 in range(0,nr_stations):
            tot_n_queue_ws[i2][i1] = 0
            for i3 in range(0,max_C):
                waiting_time_job_ws[i2][i1][i3] = 0
        
    ### VARIABLES RELATED TO Processed SCANS ### 
    for i2 in range(0,K):
        mean_system_time[i2] = 0;
        for i3 in range(0, max_C):
            time_departure[i2][i3] = 0
            time_system[i2][i3] = 0
            for i1 in range(0,nr_stations):
                time_departure_ws[i2][i1][i3] = 0
                time_system_job_ws[i2][i1][i3] = 0

    for i3 in range(0,max_C):
        order_out[i3] = 0
    
    ### OTHER PARAMETERS ###
    for i2 in range(0,K):
        for i3 in range(0,nr_stations):
            for i1 in range(0,nr_servers[i3]):
                idle[i2][i3][i1] = 0
    for i3 in range(0,nr_stations):
        rho_ws[i3] = 0
        for i1 in range(0,nr_servers[i3]):
            rho_ws_s[i3][i1] = 0

In [62]:
def init(): #Initialisation function
    
    ### SET INPUT VALUES ###
    np.random.seed(0) #((i3+1)*K-run) 
                      #Ensure you each time use a different seed to get IID replications
        
    ### INPUT RADIOLOGY DPT ###
    global nr_stations
    nr_stations= 5 #Number of workstations
    
    global nr_servers
    nr_servers={} #Input number of servers per workstation
    nr_servers[0]= 3
    nr_servers[1]= 2
    nr_servers[2]= 4
    nr_servers[3]= 3
    nr_servers[4]= 1
    
    ### INPUT JOB TYPES ###
    global nr_job_types, nr_workstations_job
    nr_job_types= 4 #Number of scans, job types 
    nr_workstations_job={} #Number of workstations per job type
    nr_workstations_job[0] = 4             
    nr_workstations_job[1] = 3
    nr_workstations_job[2] = 5
    nr_workstations_job[3] = 3
    
    global route
    route= defaultdict(dict) #Route to follow for each job type 
    route[0][0] = 2 #JOB = 1
    route[0][1] = 0
    route[0][2] = 1
    route[0][3] = 4
    
    route[1][0] = 3 #JOB = 2
    route[1][1] = 0
    route[1][2] = 2
    
    route[2][0] = 1 #JOB = 3
    route[2][1] = 4
    route[2][2] = 0
    route[2][3] = 3
    route[2][4] = 2
    
    route[3][0] = 1 #JOB = 4
    route[3][1] = 3
    route[3][2] = 4
    
    global current_station 
    current_station= {} #Matrix that denotes the current station of a scan (sequence number)
    
    ### INPUT ARRIVAL PROCESS ###
    global nr_arrival_sources
    nr_arrival_sources = 2 #Number of arrival sources
    global n_a #Number of scans arrived to the system 
    n_a= 0
    global n_a_ws, t_a, t_lambda, tot_lambda, scan_type, time_arrival, time_arrival_ws, mean_interarrival_time
    t_lambda=0
    n_a_ws= {} #Number of scans arrived to a particular workstation
    t_a= {} #Time of next arrival for each source 
    global first_ta #First arrival time over all sources 
    first_ta= 0
    global index_arr #Source of next arrival 
    index_arr=0
    tot_lambda= defaultdict(dict)
    scan_type= {} #Type of scan arriving
    time_arrival= defaultdict(dict) #Time of arrival of the scan to the ancillary services 
    time_arrival_ws= defaultdict(lambda: defaultdict(dict)) #Time of arrival of a particular scan to a particular workstation
    mean_interarrival_time= {}
    
    #Arrival from radiology department
    global lamb, cum_distr_scans
    lamb= {} #Arrival rate
    lamb[0] = 1/0.25 #Input arrival rate = 1/mean interarrival time
    cum_distr_scans= defaultdict(dict) #Cumulative(!) distribution of job types per source
    cum_distr_scans[0][0] = 0.2 #SOURCE = 1
    cum_distr_scans[0][1] = 0.4
    cum_distr_scans[0][2] = 0.5
    cum_distr_scans[0][3] = 1
    
    #Arrival from other services
    lamb[1] = 1/1 #Input arrival rate = 1/mean interarrival time
    cum_distr_scans[1][0] = 0 #SOURCE = 2
    cum_distr_scans[1][1] = 0.4
    cum_distr_scans[1][2] = 0.4
    cum_distr_scans[1][3] = 1
    
    ### INPUT SERVICE PROCESS ###
    global perc_again
    perc_again = 0.02 #The probability that a scan should be re-evaluated at a particular workstation
    global n_d #Number of scans handled
    global n_d_ws, t_d, mean_service_time, tot_mu, time_service, current_cust, list_scan
    n_d_ws= {} #Number of scans handled in a particular workstation
    t_d= defaultdict(dict) #Time of next departure for each server in each workstation
    global first_td #First departure time over all sources
    global index_dep_station #Station with first departure
    global index_dep_server #Server with first departure
    mean_service_time= {} #Calculated average service time
    global t_mu #Generated service time
    n_d= first_td= index_dep_station= index_dep_server= t_mu= 0
    tot_mu= {} #Total service time generated
    time_service= defaultdict(lambda: defaultdict(dict)) #Service time per customer and workstation
    current_cust= defaultdict(dict) #Customer handles by a particular workstation and server
    list_scan= defaultdict(dict) #list of customers processed at a particular workstation on a particular point in time
    
    global mu
    mu= defaultdict(dict) #Processing time per ws and job type
    mu[0][0] = 12 #WS1, J1
    mu[0][1] = 15
    mu[0][2] = 15
    mu[0][3] = 0
    mu[1][0] = 20 
    
    mu[1][1] = 0 #WS2, J1
    mu[1][2] = 21
    mu[1][3] = 18
    mu[2][0] = 16
    
    mu[2][1] = 14 #WS3, J1
    mu[2][2] = 10
    mu[2][3] = 0
    
    mu[3][0] = 0 #WS4, J1
    mu[3][1] = 20
    mu[3][2] = 24
    mu[3][3] = 13
    
    mu[4][0] = 25 #WS5, J1
    mu[4][1] = 0
    mu[4][2] = 20
    mu[4][3] = 25
    
    global var
    var= defaultdict(dict) #Processing variance per ws and job type
    var[0][0] = 2 #WS1, J1
    var[0][1] = 2
    var[0][2] = 3
    var[0][3] = 0
    
    var[1][0] = 4 #WS2, J1
    var[1][1] = 0
    var[1][2] = 3
    var[1][3] = 3
    
    var[2][0] = 4 #WS3, J1
    var[2][1] = 2
    var[2][2] = 1
    var[2][3] = 0
    
    var[3][0] = 0 #WS4, J1
    var[3][1] = 3
    var[3][2] = 4
    var[3][3] = 2
    
    var[4][0] = 5 #WS5, J1
    var[4][1] = 0
    var[4][2] = 3
    var[4][3] = 5
    
    global sigma
    sigma= defaultdict(dict) #Processing stdev per ws and job type
    sigma[0][0] = np.sqrt(var[0][0]) #WS1, J1
    sigma[0][1] = np.sqrt(var[0][1])
    sigma[0][2] = np.sqrt(var[0][2])
    sigma[0][3] = np.sqrt(var[0][3])
    
    sigma[1][0] = np.sqrt(var[1][0]) #WS2, J1
    sigma[1][1] = np.sqrt(var[1][1])
    sigma[1][2] = np.sqrt(var[1][2])
    sigma[1][3] = np.sqrt(var[1][3])
    
    sigma[2][0] = np.sqrt(var[2][0]) #WS3, J1
    sigma[2][1] = np.sqrt(var[2][1])
    sigma[2][2] = np.sqrt(var[2][2])
    sigma[2][3] = np.sqrt(var[2][3])
    
    sigma[3][0] = np.sqrt(var[3][0]) #WS4, J1
    sigma[3][1] = np.sqrt(var[3][1])
    sigma[3][2] = np.sqrt(var[3][2])
    sigma[3][3] = np.sqrt(var[3][3])
    
    sigma[4][0] = np.sqrt(var[4][0]) #WS5, J1
    sigma[4][1] = np.sqrt(var[4][1])
    sigma[4][2] = np.sqrt(var[4][2])
    sigma[4][3] = np.sqrt(var[4][3])
    
    ### GENERAL DISCRETE EVENT SIMULATION PARAMETERS ###
    global N
    N= 1000 #Number of scans (Stop criterion)
    global t #Simulation time
    t=1
    
    ### VARIABLES RELATED TO system SCANS ###
    global n #Number of scans in the system
    n=0
    global n_ws, mean_customers_system, tot_n, tot_n_ws
    n_ws= {} #Number of scans at a particular workstation 
    mean_customers_system= {} 
    tot_n= {} #Number of customers in the system over time
    tot_n_ws= defaultdict(dict) #Number of customers in a workstation over time 
    
    ### VARIABLES RELATED TO waiting OF SCANS ###
    global mean_waiting_time, waiting_time, waiting_time_job_ws, mean_customers_queue, tot_n_queue, tot_n_queue_ws
    mean_waiting_time= {}
    waiting_time= {}
    waiting_time_job_ws= defaultdict(lambda: defaultdict(dict)) #Waiting time for a job on a particular workstation

    mean_customers_queue= {}
    tot_n_queue= {}
    tot_n_queue_ws= defaultdict(dict) #Total number of scans in queue at workstation over time
    
    ### VARIABLES RELATED TO Processed SCANS ###
    global time_departure, time_departure_ws, time_system, time_system_job_ws, order_out, mean_system_time
    time_departure= defaultdict(dict)
    time_departure_ws= defaultdict(lambda: defaultdict(dict))

    time_system= defaultdict(dict)
    time_system_job_ws= defaultdict(lambda: defaultdict(dict))

    order_out= {}
    mean_system_time= {}
    
    ### OTHER PARAMETERS ###
    global infinity, idle, rho_ws_s, rho_ws, rho
    infinity= rho=0
    idle= defaultdict(lambda: defaultdict(dict))
    rho_ws_s= defaultdict(dict)
    rho_ws= {}
    
    ### VARIABLES RELATED TO CLOCK TIME ###
    global elapsed_time, time_subproblem, start_time, inter_time, project_start_time
    
    ### PUT ALL VARIABLES TO ZERO ###
    initialize_functions()
    
    ### INITIALISE SYSTEM ###
    
    ### DETERMINE FIRST ARRIVAL AND FIRST DEPARTURE ###
    # TO DO STUDENT    # Put all departure times for all customers to +infty
    # TO DO STUDENT    # Generate first arrival for all sources
    # TO DO STUDENT    # Get next arrival
    # TO DO STUDENT    # Calculate average arrival time to the system

In [63]:
def arrival_event():
    # TO DO STUDENT        # DEFINE ARRIVAL EVENT
    print("test")

In [64]:
def departure_event():
    # TO DO STUDENT        # DEFINE DEPARTURE EVENT
    print("test")

In [65]:
def radiology_system():
    # TO DO STUDENT        # Perform simulation until prespecified number of customers have departed (while loop)
    
    # TO DO STUDENT        # Identify next departure event
        
    # TO DO STUDENT        # Identify next arrival event
    
    # TO DO STUDENT        # Identify next event (arrival or departure)
    
    # TO DO STUDENT        # ARRIVAL EVENT
    
    # TO DO STUDENT        # DEPARTURE EVENT
    print("test")

In [66]:
def output():
    global rho #waarom werkt dit niet als global hier niet staat? 
    file1 = open("Output_Radiology.txt","w")
    for i1 in range(0, nr_stations):
        file1.write('Utilisation servers Station WS %d:\t' % i1)
        for i2 in range(0, nr_servers[i1]):
            j1= idle[run][i1][i2]/t     
            rho_ws_s[i1][i2]= 1-j1
            file1.write('%lf\t' % rho_ws_s[i1][i2])
        file1.write('\n')
    file1.write('\n')
    for i1 in range(0,nr_stations):
        file1.write('Avg utilisation Station WS %d:\t' % i1)
        for i2 in range(0, nr_servers[i1]):
            rho_ws[i1] += rho_ws_s[i1][i2]
        rho_ws[i1]= rho_ws[i1]/nr_servers[i1]
        file1.write('%lf\n' % rho_ws[i1])
    file1.write('\n')

    for i1 in range(0, nr_stations):
        rho += rho_ws[i1]
    rho /= nr_stations
    file1.write('Overall avg utilisation: %lf\n' % rho)
    file1.write('\n')
    
    rho= rho/nr_stations

    for i1 in range(0,N): #PRINT system time = cycle time (observations and running average)
        mean_system_time[run] += time_system[run][order_out[i1]]
    file1.write('Cycle time\n\n')
    j1 = mean_system_time[run]/N
    file1.write('Avg cycle time: %lf\n\n' % j1)
    
    mean_system_time[run] = 0
    file1.write('Number\tObservation\tRunning Average\n')

    for i1 in range(0,N):
        mean_system_time[run] += time_system[run][order_out[i1]]
        j1 = mean_system_time[run]/(i1+1)
        file1.write('%d\t' % i1 + '%lf\t' % time_system[run][order_out[i1]] + '%lf\n'% j1)

In [67]:
L=1
for i3 in range(0,L):
    K=1
    for run in range(0,K):
        init()
        radiology_system()
        output()

test
